In [1]:
%reload_ext autoreload

In [34]:
#!pip install wandb

In [35]:
# Imports

%load_ext autoreload
%autoreload 2

%matplotlib inline
import numpy as np
import random
import csv
import pandas as pd
import matplotlib.pyplot as plt
import wandb

from fastai.imports import *
from fastai.structured import *
from matplotlib import pyplot
from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display
from tabulate import tabulate
from sklearn.model_selection import GridSearchCV
from scipy.stats import trim_mean, kurtosis
from scipy.stats.mstats import mode, gmean, hmean
from imblearn.over_sampling import SMOTE
from sklearn import metrics
from sklearn.externals import joblib
from sklearn.metrics import f1_score,\
    accuracy_score, confusion_matrix,\
    precision_score, recall_score,\
    roc_curve, roc_auc_score,\
    cohen_kappa_score, mean_absolute_error,\
    precision_recall_curve, auc,\
    average_precision_score

set_plot_sizes(12,14,16)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import csv

sys.path.append('../lib')
import utils
import preprocessing

In [4]:
# Load in data

PATH = "../data/"
df = pd.read_csv(f'{PATH}churn.csv', low_memory=False, 
                     parse_dates=['canceldate', 'licence_registration_date', 'golive_date'])

In [5]:
df.columns

Index(['username', 'licence_registration_date', 'licence_status', 'url',
       'golive_date', 'canceldate', 'plan', 'days_active', 'account_status',
       'customer_account_status', 'nps', 'last_login_days', 'golive_days',
       'last_month_total', 'mtd_total', 'last_month_open', 'mtd_open',
       'last_month_closed', 'mtd_closed', 'last_month_age_hours',
       'mtd_age_hours', 'month_total_avg', 'open_month_avg',
       'closed_month_avg', 'age_hours_month_avg', 'interactions_total',
       'interactions_completed', 'interactions_no_response',
       'interactions_no_onboarding', 'interactions_completed_training',
       'churn_concern_cases_total', 'churn_concern_cases_open',
       'total_churn_concern_cases_age', 'total_churn_concern_cases_age_avg',
       'churned'],
      dtype='object')

In [6]:
# Delete rubbish rows
rubbish_urls = ['tobias-tattoo-a.neto.com.au', 'www.lovesongs.com.au', 'vantage-demo.neto.com.au', 'marshall.neto.com.au']

for url in rubbish_urls:
    loc = df[df.url == url].index[0]
    df = df.drop([loc], axis=0)

drop_columns = ['url', 'golive_date']

df_raw = df.drop(columns=drop_columns)
    
# Delete rubbish usernames
rubbish_usernames = ['c8kLmHScrYaUKAz']

for username in rubbish_usernames:
    loc = df_raw[df_raw.username == username].index[0]
    df_raw = df_raw.drop([loc], axis=0)

In [7]:
print(len(df_raw.loc[df_raw.churned == 0]), df_raw.loc[df_raw.churned == 0].shape)

df_processed = preprocessing.prepare_data(df_raw.loc[(df_raw.licence_status != "STAGING") & (df_raw.licence_status != "PARTNER") & (df_raw.plan != "CUSTOM") & (df_raw.plan != "TRIAL")])
print(len(df_processed), df_processed.shape)

2709 (2709, 33)
Sorting dataframe by licence_registration_date ...
Defaulting 'last_login_days' for missing values to the plans mean or platform medeian ...
  Platform 'last_login_days' median: 12.0
  Platform 'last_login_days' mean: 17.24123362906633


../lib/preprocessing.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['plan'][i] = "no_plan"
../lib/preprocessing.py:129: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['plan_not_set'][i] = 1


  List records that still have NaN in them for feature 'last_login_days' ...
  0
Defaulting 'last_month_total' for missing values to the plans mean or platform medeian ...
  Platform 'last_month_total' median: 0.0
  Platform 'last_month_total' mean: 1.5262305699481866
  List records that still have NaN in them for feature 'last_month_total' ...
  0
Defaulting 'mtd_total' for missing values to the plans mean or platform medeian ...
  Platform 'mtd_total' median: 0.0
  Platform 'mtd_total' mean: 0.234779792746114
  List records that still have NaN in them for feature 'mtd_total' ...
  0
Defaulting 'last_month_open' for missing values to the plans mean or platform medeian ...
  Platform 'last_month_open' median: 0.0
  Platform 'last_month_open' mean: 0.21275906735751296
  List records that still have NaN in them for feature 'last_month_open' ...
  0
Defaulting 'mtd_open' for missing values to the plans mean or platform medeian ...
  Platform 'mtd_open' median: 0.0
  Platform 'mtd_open' me

../lib/preprocessing.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[feature][i] = random_replace_value(df, feature)


Feature has NaN values: licence_registration_Year, 116
Feature has NaN values: licence_registration_Month, 116
Feature has NaN values: licence_registration_Week, 116
Feature has NaN values: licence_registration_Day, 116
Feature has NaN values: licence_registration_Dayofweek, 116
Feature has NaN values: licence_registration_Dayofyear, 116
5903 (5903, 101)


In [8]:
print(len(df_processed), df_processed.shape)
df_processed = df_processed.loc[df_processed.licence_status == "LIVE"]
df_processed = df_processed.drop(columns=['licence_status'])
print(len(df_processed), df_processed.shape)

5903 (5903, 101)
2380 (2380, 100)


In [9]:
print(len(df_processed), df_processed.shape)

df_data, y_data, nas = proc_df(df_processed, 'churned')

print(len(df_data))
print(df_data.shape)
print(nas)

2380 (2380, 100)
2380
(2380, 106)
{'golive_days': 91.0, 'licence_registration_Year': 2017.0, 'licence_registration_Month': 6.0, 'licence_registration_Week': 26.0, 'licence_registration_Day': 19.0, 'licence_registration_Dayofweek': 2.0, 'licence_registration_Dayofyear': 182.0}


In [10]:
# this is where we "checkout" our latest model, likely done at container build time
joblib_file = "./wandb/churn_model.pkl"

# Load from file
churn_model = joblib.load(joblib_file)

In [11]:
predictions = churn_model.predict(df_data)
probability = churn_model.predict_proba(df_data)[:,1]
len(probability), len(predictions), len(df_processed)

(2380, 2380, 2380)

In [12]:
df_predictions = pd.DataFrame(predictions)
df_probability = pd.DataFrame(probability)

In [13]:
#df_output = df_processed[df_processed.churned == 0].copy()
#df_output = df_processed[df_processed.churned == 0].copy()

In [14]:
#df_output['prediction'] = predictions
#df_output['probability'] = probability
#df_output.prediction, y_data

df_data['prediction'] = predictions
df_data['probability'] = probability

accuracy_score(y_data,df_data['prediction'])

#len(y_data), len(df_data['prediction'])
#y_data[:20], df_data
#df_data[y_data != df_data.prediction]

0.988655462184874

In [15]:
# Calculate churn concern %
churn_concern_count = len(df_data.loc[df_data['prediction'] == 1])
total_active = len(df_data)
churn_concern_percent = round((churn_concern_count / total_active) * 100, 2)
print(f"Churn concern count: {churn_concern_count} of {total_active} ({churn_concern_percent}%)")

Churn concern count: 27 of 2380 (1.13%)


In [16]:
# Sort churn concerns by probability
df_sorted = df_data.loc[df_data['prediction'] == 1].sort_values(by=['probability'], ascending=False)

In [17]:
#df_output.annual_revenue_log.var(), df_output.annual_revenue_log.skew(), df_output.annual_revenue_log.mad(), df_output.annual_revenue_log.mode()



In [18]:
df_sorted.shape, df_processed.shape

((27, 108), (2380, 100))

In [19]:
index_number = 6268
df_processed.loc[df_processed.index == index_number]

,username,last_login_days,golive_days,last_month_total,mtd_total,last_month_open,mtd_open,last_month_closed,mtd_closed,last_month_age_hours,...,licence_registration_Day,licence_registration_Dayofweek,licence_registration_Dayofyear,licence_registration_Is_month_end,licence_registration_Is_month_start,licence_registration_Is_quarter_end,licence_registration_Is_quarter_start,licence_registration_Is_year_end,licence_registration_Is_year_start,licence_registration_Elapsed


In [20]:
df_data.loc[df_data.index == index_number]

,username,last_login_days,golive_days,last_month_total,mtd_total,last_month_open,mtd_open,last_month_closed,mtd_closed,last_month_age_hours,...,licence_registration_Elapsed,golive_days_na,licence_registration_Year_na,licence_registration_Month_na,licence_registration_Week_na,licence_registration_Day_na,licence_registration_Dayofweek_na,licence_registration_Dayofyear_na,prediction,probability


In [21]:
df_sorted.loc[df_sorted.index == index_number]

,username,last_login_days,golive_days,last_month_total,mtd_total,last_month_open,mtd_open,last_month_closed,mtd_closed,last_month_age_hours,...,licence_registration_Elapsed,golive_days_na,licence_registration_Year_na,licence_registration_Month_na,licence_registration_Week_na,licence_registration_Day_na,licence_registration_Dayofweek_na,licence_registration_Dayofyear_na,prediction,probability


In [41]:
!wandb login dda97ba4df7f598b91affe3c806891df2e043ed7

wandb: Appending key for api.wandb.ai to your netrc file: /Users/justinhennessy/.netrc
Successfully logged in to Weights & Biases!


In [42]:
import time
seconds = time.time()

run_name = f"customer-inference-{int(seconds)}"
print(run_name)

wandb.init(project="neto-customer-churn", name=f"customer-inference-{int(seconds)}")


now = datetime.datetime.now()
array = []

for index, row in df_sorted.iterrows():
    output_string = f"{now}, " \
                    f"{df_processed.username.loc[df_processed.index == index].values[0]}, " \
                    f"{df.url.loc[df.index == index].values[0]}, " \
                    f"{df_raw.plan.loc[df_raw.index == index].values[0]}, , " \
                    f"{row.probability}, , " \
                    f"{df_raw.licence_status.loc[df_raw.index == index].values[0]}"
    print(output_string)
    array.append([output_string])
    
print(array)

with open('./wandb/predictions.csv', 'w') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerows(array)

writeFile.close()
wandb.save("predictions.csv")

customer-inference-1573417068


TypeError: super(type, obj): obj must be an instance or subtype of type

In [190]:
n_number = 'N060915'
utils.display_all(df_processed[df_processed.username == n_number])

,username,last_login_days,golive_days,last_month_total,mtd_total,last_month_open,mtd_open,last_month_closed,mtd_closed,last_month_age_hours,mtd_age_hours,month_total_avg,open_month_avg,closed_month_avg,age_hours_month_avg,churned,plan_not_set,days_active_modified,customer_account_status_PoorStanding,plan_BASIC,plan_BRONZE,plan_BUSINESS,plan_BUSINESS +,plan_CUSTOM XL,plan_ENTERPRISE,plan_GOLD,plan_GROW,plan_L&S - PREMIUM,plan_L&S - STANDARD,plan_L&S - ULTIMATE,plan_LIST N SEND SILVER,plan_LISTNSEND,plan_NETO XL,plan_NETO XXL,plan_PLAN-360-17Q2,plan_PLAN-360-19Q1,plan_PLAN-360-TRANSITION,plan_PLAN-360-US,plan_PLAN-L,plan_PLAN-L-17Q2,plan_PLAN-L-19Q1,plan_PLAN-L-US,plan_PLAN-M,plan_PLAN-M-17Q2,plan_PLAN-M-19Q1,plan_PLAN-M-US,plan_PLAN-S,plan_PLAN-S-17Q2,plan_PLAN-S-19Q1,plan_PLAN-S-US,plan_PLAN-XL-19Q1,plan_PLAN-XS,plan_PLAN-XS-17Q2,plan_PREMIER,plan_PREMIUM,plan_SILVER,plan_STANDARD,plan_START,plan_START - LEGACY,plan_TELSTRA - ONLINE STORE M,plan_TELSTRA - ONLINE STORE S,plan_TELSTRA - POS S,plan_TELSTRA - PREMIUM,plan_TELSTRA - STANDARD,plan_ULTIMATE,plan_ULTIMATE PLUS,plan_no_plan,nps_no_data,nps_detractor,nps_passive,nps_promoter,last_login_categories_day,last_login_categories_few_days,last_login_categories_week,last_login_categories_fortnight,last_login_categories_month,last_login_categories_month+,days_active_no_data,days_active_day,days_active_week,days_active_fortnight,days_active_month,days_active_3months,days_active_6months,days_active_1year,days_active_2years,days_active_2years+,licence_registration_Year,licence_registration_Month,licence_registration_Week,licence_registration_Day,licence_registration_Dayofweek,licence_registration_Dayofyear,licence_registration_Is_month_end,licence_registration_Is_month_start,licence_registration_Is_quarter_end,licence_registration_Is_quarter_start,licence_registration_Is_year_end,licence_registration_Is_year_start,licence_registration_Elapsed
5987,N060915,12.0,58.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,2019.0,3.0,11.0,13.0,2.0,72.0,False,False,False,False,False,False,1552435200


In [191]:
from eli5 import show_prediction
loc_address = df_processed.index[df_processed.username == n_number].values[0]
#df_data = df_data.drop(columns=['prediction', 'probability'])
show_prediction(churn_model, df_data.loc[loc_address], show_feature_values=True)